# This notebook extracts airborne data closest to pits, plots ground data and SMRT simulations

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import math
import matplotlib.pyplot as plt
from scipy.stats import linregress
from numpy.polynomial.polynomial import polyfit
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
%matplotlib widget

In [ ]:
%load_ext autoreload
%aimport aesop
%aimport run_smrt_with_realistic_atmosphere
%autoreload 1

Load pits:

In [ ]:
pits = aesop.get_pit_topography()
for pit in pits.index:
    for aoi in ['A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09']:
        if aoi in pit:
            pits.loc[pit, 'aoi'] = aoi
    if pit == 'MetStation':
        pits.loc[pit, 'aoi'] = 'A04'

Uncorrected ground data:

In [ ]:
sbr = pd.read_csv('../Data/ground_obs/more89forMel.csv')

In [ ]:
tb89v0 = aesop.ground_based_data(sbr, '89V0', pits)

Load SMRT simulations and correct to ground height for comparison with ground based radiometer observations. Use simulations for both flights.

In [ ]:
smrt_c087 = xr.open_dataset('../Other/smrt_results_atmos/smrt_atmos_tb_C087.nc')
smrt_c090 = xr.open_dataset('../Other/smrt_results_atmos/smrt_atmos_tb_C090.nc')

In [ ]:
def correct_to_surface(smrt_tb, flight):
    
    offset = aesop.bt_offset('M16-89')
    
    corr = []
    for pit in smrt_tb.snowpack:
        if str(pit.values) == 'MetS':
            aoi = 'A04'
        else:
            aoi = str(pit.values)[0:3]

        atmos = run_smrt_with_realistic_atmosphere.get_atmosphere(flight, aoi, True)
        tbup, trans = atmos.tbup(89e9,1.0,2)[0], atmos.trans(89e9,1.0,2)[0]

        corr.append((((smrt_tb.sel(snowpack=pit, channel='M16-89')+offset)-tbup)/trans)-offset)

    return xr.concat(corr, dim='snowpack')

In [ ]:
c087_corrected = correct_to_surface(smrt_c087, 'C087').loc[{'snowpack': pits.index}]
c090_corrected = correct_to_surface(smrt_c090, 'C090').loc[{'snowpack': pits.index}]

Plot SMRT simulations vs ground based observations at nadir:

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

smrt_base = np.mean([c087_corrected['base'].values, c090_corrected['base'].values], axis=0)
smrt_high = np.mean([c087_corrected['high'].values, c090_corrected['high'].values], axis=0)
smrt_low = np.mean([c087_corrected['low'].values, c090_corrected['low'].values], axis=0)

bars_a = ax.errorbar(tb89v0[:,0], smrt_base,
             xerr=[tb89v0[:,1],tb89v0[:,2]],
             yerr=[smrt_base-smrt_low,smrt_high-smrt_base],
             fmt='o', label='89V0', color='b', alpha=0.6)
[bar.set_alpha(0.6) for bar in bars_a[2]]


axins = zoomed_inset_axes(ax, 2, loc=4) # zoom = 6
axins.plot(tb89v0[:,0], smrt_base, 'bo', alpha=0.6)
axins.set_xlim(176, 198) # Limit the region for zoom
axins.set_ylim(142, 165)

axins.set_xticks([])  # Not present ticks
axins.set_yticks([])

## draw a bbox of the region of the inset axes in the parent axes and
## connecting lines between the bbox and the inset axes area
mark_inset(ax, axins, loc1=2, loc2=3, fc="none", ec="0.5", alpha=0.2) 

for label, x, y in zip(pits.index, tb89v0[:,0], smrt_base):
    if label in ['A03W', 'A05W']:
        ax.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")
    if label in ['A03C1', 'A06N']:
        axins.annotate(label, xy=(x, y), xytext=(3, 3), textcoords="offset points")
    elif label in ['A06S1']:
        axins.annotate(label, xy=(x, y), xytext=(3, -12), textcoords="offset points")
        

ax.plot(np.arange(100, 280, 10), np.arange(100, 280, 10), 'k--')

# plt.legend()
# plt.title('SMRT vs SBR @89 GHz')
ax.set_xlabel('Ground Observed TB [K]')
ax.set_ylabel('SMRT TB [K]')
ax.tick_params(axis='x')
ax.tick_params(axis='y')

ax.set_aspect('equal', adjustable='box')
ax.set_xlim(120, 260)
ax.set_ylim(120, 260)
fig.canvas.width = '600px'
fig.canvas.height = '600px'
#fig.savefig('../Figs/Fig4.png')

In [ ]:
x = tb89v0[:,0]
y = smrt_base
mask = ~np.isnan(x)

slope, intercept, r_value, p_value, std_err = linregress(x[mask], y[mask])
print ('r^2: ',r_value**2)
print ('ME: ', np.sum(y[mask]-x[mask]) / len(y[mask]))
print ('rmse: ', np.sqrt(np.mean((y[mask]-x[mask])**2)))

In [ ]:
# Remove outliers A06N [18], A06S1 [19] and A03C1 [3]
import copy
remask = copy.deepcopy(mask)

In [ ]:
remask[3] = False #A03C1
remask[18] = False #A06N
remask[19] = False #A06S1

In [ ]:
newslope, newintercept, newr_value, newp_value, newstd_err = linregress(x[remask], y[remask])
print ('r^2: ',newr_value**2)
print ('ME: ', np.sum(y[remask]-x[remask]) / len(y[remask]))
print ('rmse: ', np.sqrt(np.mean((y[remask]-x[remask])**2)))